In [1]:
import pandas as pd
from datetime import timedelta
import os
from typing import List, Dict, Any
import json

os.chdir('../')

from src.data_handler import *

In [2]:
Y = 2021
W = 32
path = f'data/to_process/maintenance_data_{Y}-{W:02d}.xlsx'

df_example = read_data(path)
df_example.head()

,Equipos,Horómetro,Fecha_Actual,Hora_de_Detención,Hora_de_Entrega,Tiempo_FS,Nro_Personas,Horas_Persona,Sistema,Sub_Sistemas,Tipo_de_Detención,Trabajo_Ejecutado,Nº_OT,Nº_Backlog,Status_Final,Trabajos_Pendientes,Jefe_de_Turno,Turnos
0,T_09,86075,2021-08-09,10:42:00,10:51:00,0.15,2,0.30,T_Engrase,TK Grasa,Operacional,ot 402803 se asiste a equipo y se rellena tk d...,0,NaN,Operativo,NaN,jorge pasten,Dia
1,T_09,86075,2021-08-09,10:51:00,12:18:00,1.45,0,0.00,T_Neumaticos,Inspección,Operacional,se realiza chequeo preventivo por llamado oper...,0,NaN,Operativo,NaN,jorge pasten,Dia
2,T_11,88951,2021-08-08,03:50:00,04:30:00,0.67,2,1.34,T_Motor,Filtros,Stand-By,ot-402797 operador informa alarma de presión d...,402797,NaN,Operativo,NaN,Hernán Veliz,Noche
3,T_11,88958,2021-08-09,13:31:00,16:19:00,2.80,2,5.60,T_Engrase,TK Grasa,Falla,c.s.i.se acude a equipo por alarma de lubricac...,0,NaN,Operativo,NaN,jorge pasten,Dia
4,T_11,89049,2021-08-14,09:35:00,09:45:00,0.17,2,0.34,T_Hidraulico,TK Hidraulico,Operacional,se realiza relleno de aceite hidráulico implem...,0,NaN,Operativo,NaN,Manuel González,Dia


In [3]:
df_example_processed = read_and_process_data(path, Y, W)
df_example_processed.head()

,UnitId,Date,start_time,end_time,time_FS,System,Subsystem,type_detention,observation,observation_length,observation_length_new,isMantention,empty_obs
0,T_09,2021-08-09,2021-08-09 10:42:00,2021-08-09 10:51:00,0.15,Engrase,TK Grasa,Operacional,ot 402803 se asiste a equipo y se rellena tk d...,71,71,No.,False
1,T_09,2021-08-09,2021-08-09 10:51:00,2021-08-09 12:18:00,1.45,Neumaticos,Inspección,Operacional,se realiza chequeo preventivo por llamado oper...,450,450,No.,False
2,T_11,2021-08-08,2021-08-08 03:50:00,2021-08-08 04:30:00,0.67,Motor,Filtros,Stand-By,ot-402797 operador informa alarma de presion d...,290,290,No.,False
3,T_11,2021-08-09,2021-08-09 13:31:00,2021-08-09 16:19:00,2.80,Engrase,TK Grasa,Falla,se acude a equipo por alarma de lubricacion- s...,819,819,No.,False
4,T_11,2021-08-14,2021-08-14 09:35:00,2021-08-14 09:45:00,0.17,Hidraulico,TK Hidraulico,Operacional,se realiza relleno de aceite hidraulico implem...,57,57,No.,False


In [4]:
import plotly.express as px

# Calculate main statistics
mean_orig = df_example_processed['observation_length'].mean()
mean_new = df_example_processed['observation_length_new'].mean()
median_orig = df_example_processed['observation_length'].median()
median_new = df_example_processed['observation_length_new'].median()
std_orig = df_example_processed['observation_length'].std()
std_new = df_example_processed['observation_length_new'].std()

print("Observation Length Statistics:")
print(f"Original: mean={mean_orig:.2f}, median={median_orig:.2f}, std={std_orig:.2f}")
print(f"New     : mean={mean_new:.2f}, median={median_new:.2f}, std={std_new:.2f}")

df_plot = pd.DataFrame({
    "Type": ["Original Length"] * len(df_example_processed) + ["New Length"] * len(df_example_processed),
    "Observation Length": pd.concat([
        df_example_processed["observation_length"],
        df_example_processed["observation_length_new"]
    ], ignore_index=True)
})

fig = px.violin(
    df_plot,
    x="Observation Length",
    y="Type",
    color="Type",
    box=True,
    points=False,
    orientation="h",
    color_discrete_map={"Original Length": "#1f77b4", "New Length": "#ff7f0e"},
    template="plotly_white"
)
fig.update_layout(
    title="Comparison of Observation Lengths (Violin Plot)",
    font=dict(size=14),
    plot_bgcolor='rgba(245,245,245,1)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    xaxis_title="Observation Length",
    yaxis_title=""
)
fig.show()

Observation Length Statistics:
Original: mean=311.10, median=163.00, std=318.44
New     : mean=291.96, median=155.50, std=333.02


In [5]:
# Insert a newline every 150 characters in observation for readability
def insert_newlines(text, every=120):
    return '\n'.join([text[i:i+every] for i in range(0, len(text), every)])


df_example_processed.sort_values(by='observation_length', ascending=False, inplace=True)

row = df_example_processed.iloc[7]
row2 = df_example[(df_example.Equipos == row.UnitId) & (df_example.Fecha_Actual == row.Date)].iloc[1]

c = row.observation
c2 = row2.Trabajo_Ejecutado

print("Example of a long observation with newlines inserted for readability:")
print(insert_newlines(c))

print("Example of a long observation from the original data:")
print(insert_newlines(c2))

Example of a long observation with newlines inserted for readability:
Mantenimiento Programado.
ot: 381397. se conectan ductos de admision y escape. se cambia union de turbo delantero izquie
rdo. se instala mantas termicas. se retira arnes de motor diesel. se instala arnes de motor diesel y se rutea arnes. se 
elimina fuga por tubo succion direccion correspondiente a ot: 381397. se cambia conector de inyector numero 1. se rellen
a aceite hidraulico direccion 144 lts. l.cisterna. 
ot: 401642. se realiza chequeo de compresor, correas y mangueras. pr
esiones. se realiza cambio de termostato, presostato de alta y baja. cambio de filtros de cabina.
. se realiza cambio de
 difusores, funda, sello puerta, ajuste y limpieza de chapas. chequeo de asiento y suspension. chequeo de cortina, radio
 musical, espejo panoramicos y chequeo electrico de cabina.
Example of a long observation from the original data:
en continuacion de trabajos asociados a 1/3 vida de motor diesel y trabajos pendientes, se

In [6]:
Y = 2024
W = 25
path = f'data/to_process/maintenance_data_{Y}-{W:02d}.xlsx'

df_example = read_data(path)
df_example_processed = read_and_process_data(path, Y, W)
df_example_processed.sort_values(by='observation_length', ascending=False, inplace=True)

row = df_example_processed.iloc[7]
row2 = df_example[(df_example.Equipos == row.UnitId) & (df_example.Fecha_Actual == row.Date)].iloc[1]

c = row.observation
c2 = row2.Trabajo_Ejecutado

print("Example of a long observation with newlines inserted for readability:")
print(insert_newlines(c))

print("Example of a long observation from the original data:")
print(insert_newlines(c2))

Example of a long observation with newlines inserted for readability:
Mantenimiento Programado.
se comienza con mantencion programada pm 500 con numero de ot:520219 se posiciona equipo en lo
za .se drena y rellena aceite de mazas 44 litros, se realiza cambio de filtro de motor transmision, convertidor, diferen
cial y combustible. se realiza limpieza de conectores segun pauta. se realiza de conectores segun pauta, se realiza camb
io de sellos segun pauta. se reprograman backlog. se comienza con predictivo ot:520221 se realiza toma de todas las mues
tras y se rotulan. toma de fotografias y chequeo de rejillas y tapones magneticos. se realiza dialisis de hidraulico. ma
ntener en observacion posicion 1 por fuga por sello espejo. se realiza ot:538702 por el cambio de sellos de conectores y
 fitting valvula control. se cambio sellos a tapon valvula control.––––
Example of a long observation from the original data:
se comienza con mantencion programada pm 500 con numero de ot:520219 se posici

In [7]:
Y = 2022
W = 8
path = f'data/to_process/maintenance_data_{Y}-{W:02d}.xlsx'

df_example = read_data(path)
df_example_processed = read_and_process_data(path, Y, W)
df_example_processed.sort_values(by='observation_length', ascending=False, inplace=True)

row = df_example_processed.iloc[7]
row2 = df_example[(df_example.Equipos == row.UnitId) & (df_example.Fecha_Actual == row.Date)].iloc[0]

c = row.observation
c2 = row2.Trabajo_Ejecutado

print("Example of a long observation with newlines inserted for readability:")
print(insert_newlines(c))

print("\n\nExample of a long observation from the original data:")
print(insert_newlines(c2))

ValueError: No data available for the specified year and week.

In [ ]:
c

'Mantenimiento Programado.\not: 347863. cambio de front corner izquierdo. se retiran mangueras de enfriamiento freno izquierdo y se fabrican nuevas en emsesa con 15 cm. adicional. se rellena aceite hidraulico implemento 150 lts. y aceite a masa izquierda 22 lts. se carga suspension con pulgada de aceite y 9 pulgadas de nitrogeno. se cambia valvula de carga suspension.\npersonal , realiza instalacion de neumatico posicion 01 segun procedimiento. '

In [8]:
Y = 2025
W = 29
path = f'data/to_process/maintenance_data_{Y}-{W:02d}.xlsx'

df_example = read_data(path)
df_example_processed = read_and_process_data(path, Y, W)
df_example_processed.sort_values(by='observation_length', ascending=False, inplace=True)

row = df_example_processed.iloc[7]
row2 = df_example[(df_example.Equipos == row.UnitId) & (df_example.Fecha_Actual == row.Date)].iloc[0]

c = row.observation
c2 = row2.Trabajo_Ejecutado

print("Example of a long observation with newlines inserted for readability:")
print(insert_newlines(c))

print("\n\nExample of a long observation from the original data:")
print(insert_newlines(c2))

Example of a long observation with newlines inserted for readability:
Mantenimiento Programado.



Example of a long observation from the original data:
trabajos asociados a rebuilding por parte de finning


In [10]:
df_example

,Equipos,Horómetro,Fecha_Actual,Hora_de_Detención,Hora_de_Entrega,Tiempo_FS,Nro_Personas,Horas_Persona,Sistema,Sub_Sistemas,Tipo_de_Detención,Trabajo_Ejecutado,Nº_OT,Nº_Backlog,Status_Final,Trabajos_Pendientes,Jefe_de_Turno,Turnos
0,T_09,109967,2025-07-20,07:00:00,19:00:00,12.00,0,0.00,T_Mantencion,Mant. Mayor,Programada,trabajos asociados a rebuilding por parte de f...,0,NaN,En Reparacion/Mantencion,NaN,Francisco Ogalde,Dia
1,T_17,100897,2025-07-20,07:00:00,19:00:00,12.00,3,36.00,T_Mantencion,Mant. Mayor,Programada,se comienza con búsqueda de arnes del diferenc...,0,NaN,En Reparacion/Mantencion,Personal maestranza mys rematar trabajo de rep...,Francisco Ogalde,Dia
2,T_15,98563,2025-07-20,09:02:00,09:25:00,0.38,2,0.76,T_Mantencion,Relleno de Fluidos,Operacional,se chequea filtración aceite motor encontrándo...,0,NaN,Operativo,mantener en observación aceite motor.,Francisco Ogalde,Dia
3,T_12,110371,2025-07-20,09:30:00,09:46:00,0.27,2,0.54,T_Mantencion,Relleno de Fluidos,Operacional,se atiende equipo para relleno de aceite hidrá...,0,NaN,Operativo,mantener en observación falla transmisión.,Francisco Ogalde,Dia
4,T_24,15610,2025-07-20,17:21:00,19:00:00,1.65,0,0.00,T_Mantencion,1/3 Vida,Programada,equipo ingresa a taller por 1/3 de vida. equip...,0,NaN,En Reparacion/Mantencion,NaN,Francisco Ogalde,Dia
5,T_09,109967,2025-07-20,19:00:00,07:00:00,12.00,2,24.00,T_Mantencion,Mant. Mayor,Programada,se realiza engrase de suspensión delantera izq...,0,NaN,En Reparacion/Mantencion,NaN,Pablo Vega,Noche
6,T_17,100897,2025-07-20,19:00:00,07:00:00,12.00,0,0.00,T_Mantencion,Mant. Mayor,Programada,no se realizan trabajos durante turno noche.,0,NaN,En Reparacion/Mantencion,NaN,Pablo Vega,Noche
7,T_24,15610,2025-07-20,19:00:00,07:00:00,12.00,6,72.00,T_Mantencion,1/3 Vida,Programada,"Ot. 534170 lavado completo equipo, se realiza ...",0,NaN,En Reparacion/Mantencion,Continuar con 1/3 vida,Pablo Vega,Noche
8,T_09,109070,2025-07-21,07:00:00,19:00:00,12.00,0,0.00,T_Mantencion,Mant. Mayor,Programada,trabajos asociados a rebuilding por parte de f...,0,NaN,En Reparacion/Mantencion,NaN,Francisco Ogalde,Dia
9,T_17,100897,2025-07-21,07:00:00,19:00:00,12.00,0,0.00,T_Mantencion,Mant. Mayor,Programada,pendiente por mys maestransa,0,NaN,En Reparacion/Mantencion,NaN,Francisco Ogalde,Dia


In [13]:
df_example_processed

,UnitId,Date,start_time,end_time,time_FS,System,Subsystem,type_detention,observation,observation_length,observation_length_new,isMantention,empty_obs
9,T_24,2025-07-20,2025-07-20 19:00:00,2025-07-21 07:00:00,12.00,Mantencion,1/3 Vida,Programada,Mantenimiento Programado.\not. 534170 lavado c...,803,803,Si (1/3 Vida).,False
5,T_17,2025-07-20,2025-07-20 07:00:00,2025-07-20 19:00:00,12.00,Mantencion,Mant. Mayor,Programada,Mantenimiento Programado.\nse comienza con bus...,439,439,Si (Mant. Mayor).,False
4,T_15,2025-07-20,2025-07-20 09:02:00,2025-07-20 09:25:00,0.38,Mantencion,Relleno de Fluidos,Operacional,se chequea filtracion aceite motor encontrando...,211,211,No.,False
1,T_09,2025-07-20,2025-07-20 19:00:00,2025-07-21 07:00:00,12.00,Mantencion,Mant. Mayor,Programada,Mantenimiento Programado.\n,169,0,Si (Mant. Mayor).,True
3,T_12,2025-07-20,2025-07-20 09:30:00,2025-07-20 09:46:00,0.27,Mantencion,Relleno de Fluidos,Operacional,se atiende equipo para relleno de aceite hidra...,149,149,No.,False
8,T_24,2025-07-20,2025-07-20 17:21:00,2025-07-20 19:00:00,1.65,Mantencion,1/3 Vida,Programada,Mantenimiento Programado.\n,72,0,Si (1/3 Vida).,True
6,T_17,2025-07-20,2025-07-20 19:00:00,2025-07-21 07:00:00,12.00,Mantencion,Mant. Mayor,Programada,Mantenimiento Programado.\n,44,0,Si (Mant. Mayor).,True
2,T_09,2025-07-21,2025-07-21 07:00:00,2025-07-21 19:00:00,12.00,Mantencion,Mant. Mayor,Programada,Mantenimiento Programado.\n,42,0,Si (Mant. Mayor).,True
0,T_09,2025-07-20,2025-07-20 07:00:00,2025-07-20 19:00:00,12.00,Mantencion,Mant. Mayor,Programada,Mantenimiento Programado.\n,42,0,Si (Mant. Mayor).,True
7,T_17,2025-07-21,2025-07-21 07:00:00,2025-07-21 19:00:00,12.00,Mantencion,Mant. Mayor,Programada,Mantenimiento Programado.\n,28,0,Si (Mant. Mayor).,True
